### Solutions

#### Question 1

Write some code that generates a file containing containing rows containing the following data:

```
i, fibonacci_i, factorial_i, gcd_fibonacci_i_factorial_i
```

where:
- `i`: integer values from `0` to `100`
- `fibonacci_i`: the `i`th Fibonacci number
- `factorial_i`: the factorial of `i` (`i!`)
- `gcd_fib_i_fact_i`: the greatest common denominator of the `i`th Fibonacci number and `i!` 

Hint: look at the `math.factorial` and `math.gcd` functions in the Python docs

Also make sure to include a header row in your file.

For example, the first few rows in your file should contain this data:

```
i,fib,fact,gcd
0,1,1,1
1,1,1,1
2,2,2,2
3,3,6,3
4,5,24,1
5,8,120,8
```

##### Solution

Let's start by writing or importing the function's we'll need to calculate 
- the Fibonacci numbers (I'll use the sequence `1, 1, 2, 3, 5, ...` where the first number will be indexed as `0`. Also, we'll use the `lru_cache` decorator to speed up our recursive algorithm.
- the factorial of `i` (using the `math.factorial` function)
- the greatest common denominator (using the `math.gcd` function)

In [1]:
from functools import lru_cache
from math import factorial, gcd

In [2]:
@lru_cache
def fib(i):
    if i in {0, 1}:
        return 1
    else:
        return fib(i-1) + fib(i-2)

Let's call the `fib` function and make sure it outputs the expected results:

In [3]:
[fib(i) for i in range(10)]

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]

We could then generate the data we'll need to eventually write to a file as follows:

In [4]:
n = 10  # later we can change this to 100
for i in range(n):
    print(i, fib(i), factorial(i), gcd(fib(i), factorial(i)))

0 1 1 1
1 1 1 1
2 2 2 2
3 3 6 3
4 5 24 1
5 8 120 8
6 13 720 1
7 21 5040 21
8 34 40320 2
9 55 362880 5


Of course, we actually need this data as a single string that can be written to a file. To do this, we can `join` the string representations of each number:

In [5]:
n = 10  # later we can change this to 100
for i in range(n):
    data = [i, fib(i), factorial(i), gcd(fib(i), factorial(i))]
    row = ','.join([str(data[0]), str(data[1]), str(data[2]), str(data[3])])
    print(row)

0,1,1,1
1,1,1,1
2,2,2,2
3,3,6,3
4,5,24,1
5,8,120,8
6,13,720,1
7,21,5040,21
8,34,40320,2
9,55,362880,5


This works, but applying the `str` function to each element of `data` individually is not very elegant. 

Instead we can use the `map` function:

In [6]:
n = 10  # later we can change this to 100
for i in range(n):
    data = [i, fib(i), factorial(i), gcd(fib(i), factorial(i))]
    row = ','.join(map(str, data))
    print(row)

0,1,1,1
1,1,1,1
2,2,2,2
3,3,6,3
4,5,24,1
5,8,120,8
6,13,720,1
7,21,5040,21
8,34,40320,2
9,55,362880,5


We can now use this to write data to a file.

In [7]:
file_name = 'data.csv'

headers = ('i', 'fib' ,'fact', 'gcd')

n = 100

with open(file_name, 'w') as f:
    f.write(','.join(headers))
    f.write('\n')
    for i in range(n):
        data = [i, fib(i), factorial(i), gcd(fib(i), factorial(i))]
        row = ','.join(map(str, data))
        f.write(row)
        f.write('\n')

Go ahead and open that file in some text editor and check its content.

#### Question 2

Using the file you just generated, write three functions:
- `fib`
- `fact`
- `gcd_fib_fact`

that perform the same calculations as our original `fib` function, the `math` module's `factorial` and the `gcd` of the corresponding fibonacci and factorial numbers, but uses the data that was saved in the file as a cache/lookup mechanism - i.e. just use the numbers in the file if they are available, otherwise make the calculation.

##### Solution

The easiest approach will probably be to load up the data form the file and store it some lists that we can easily lookup.

We could do this inside each function we are going to create, but here I'm going to load up the data into our notebook, and pass the relevant data to each function - this avoids re-loading the data form file each time the function is called.

In [8]:
with open(file_name) as f:
    next(f)  # skip header row
    for row in f:
        print(row)

0,1,1,1

1,1,1,1

2,2,2,2

3,3,6,3

4,5,24,1

5,8,120,8

6,13,720,1

7,21,5040,21

8,34,40320,2

9,55,362880,5

10,89,3628800,1

11,144,39916800,144

12,233,479001600,1

13,377,6227020800,13

14,610,87178291200,10

15,987,1307674368000,21

16,1597,20922789888000,1

17,2584,355687428096000,136

18,4181,6402373705728000,1

19,6765,121645100408832000,165

20,10946,2432902008176640000,26

21,17711,51090942171709440000,1

22,28657,1124000727777607680000,1

23,46368,25852016738884976640000,46368

24,75025,620448401733239439360000,25

25,121393,15511210043330985984000000,1

26,196418,403291461126605635584000000,34

27,317811,10888869450418352160768000000,39

28,514229,304888344611713860501504000000,1

29,832040,8841761993739701954543616000000,440

30,1346269,265252859812191058636308480000000,1

31,2178309,8222838654177922817725562880000000,21

32,3524578,263130836933693530167218012160000000,2

33,5702887,8683317618811886495518194401280000000,1

34,9227465,2952327990396041408476186096435200000

So a few things:
- we should `strip` each `row`
- we'll need to split each row (on `,`), and cast each of the strings to integers

In [9]:
with open(file_name) as f:
    next(f)  # skip header row
    for row in f:
        print(row.strip())

0,1,1,1
1,1,1,1
2,2,2,2
3,3,6,3
4,5,24,1
5,8,120,8
6,13,720,1
7,21,5040,21
8,34,40320,2
9,55,362880,5
10,89,3628800,1
11,144,39916800,144
12,233,479001600,1
13,377,6227020800,13
14,610,87178291200,10
15,987,1307674368000,21
16,1597,20922789888000,1
17,2584,355687428096000,136
18,4181,6402373705728000,1
19,6765,121645100408832000,165
20,10946,2432902008176640000,26
21,17711,51090942171709440000,1
22,28657,1124000727777607680000,1
23,46368,25852016738884976640000,46368
24,75025,620448401733239439360000,25
25,121393,15511210043330985984000000,1
26,196418,403291461126605635584000000,34
27,317811,10888869450418352160768000000,39
28,514229,304888344611713860501504000000,1
29,832040,8841761993739701954543616000000,440
30,1346269,265252859812191058636308480000000,1
31,2178309,8222838654177922817725562880000000,21
32,3524578,263130836933693530167218012160000000,2
33,5702887,8683317618811886495518194401280000000,1
34,9227465,295232799039604140847618609643520000000,65
35,14930352,1033314796638614

Then split on the comma:

In [10]:
with open(file_name) as f:
    next(f)  # skip header row
    for row in f:
        print(row.strip().split(','))

['0', '1', '1', '1']
['1', '1', '1', '1']
['2', '2', '2', '2']
['3', '3', '6', '3']
['4', '5', '24', '1']
['5', '8', '120', '8']
['6', '13', '720', '1']
['7', '21', '5040', '21']
['8', '34', '40320', '2']
['9', '55', '362880', '5']
['10', '89', '3628800', '1']
['11', '144', '39916800', '144']
['12', '233', '479001600', '1']
['13', '377', '6227020800', '13']
['14', '610', '87178291200', '10']
['15', '987', '1307674368000', '21']
['16', '1597', '20922789888000', '1']
['17', '2584', '355687428096000', '136']
['18', '4181', '6402373705728000', '1']
['19', '6765', '121645100408832000', '165']
['20', '10946', '2432902008176640000', '26']
['21', '17711', '51090942171709440000', '1']
['22', '28657', '1124000727777607680000', '1']
['23', '46368', '25852016738884976640000', '46368']
['24', '75025', '620448401733239439360000', '25']
['25', '121393', '15511210043330985984000000', '1']
['26', '196418', '403291461126605635584000000', '34']
['27', '317811', '10888869450418352160768000000', '39']
['28

And finally make each item an integer - we'll use the `map` function again:

In [11]:
with open(file_name) as f:
    next(f)  # skip header row
    for row in f:
        print(list(map(int, row.strip().split(','))))

[0, 1, 1, 1]
[1, 1, 1, 1]
[2, 2, 2, 2]
[3, 3, 6, 3]
[4, 5, 24, 1]
[5, 8, 120, 8]
[6, 13, 720, 1]
[7, 21, 5040, 21]
[8, 34, 40320, 2]
[9, 55, 362880, 5]
[10, 89, 3628800, 1]
[11, 144, 39916800, 144]
[12, 233, 479001600, 1]
[13, 377, 6227020800, 13]
[14, 610, 87178291200, 10]
[15, 987, 1307674368000, 21]
[16, 1597, 20922789888000, 1]
[17, 2584, 355687428096000, 136]
[18, 4181, 6402373705728000, 1]
[19, 6765, 121645100408832000, 165]
[20, 10946, 2432902008176640000, 26]
[21, 17711, 51090942171709440000, 1]
[22, 28657, 1124000727777607680000, 1]
[23, 46368, 25852016738884976640000, 46368]
[24, 75025, 620448401733239439360000, 25]
[25, 121393, 15511210043330985984000000, 1]
[26, 196418, 403291461126605635584000000, 34]
[27, 317811, 10888869450418352160768000000, 39]
[28, 514229, 304888344611713860501504000000, 1]
[29, 832040, 8841761993739701954543616000000, 440]
[30, 1346269, 265252859812191058636308480000000, 1]
[31, 2178309, 8222838654177922817725562880000000, 21]
[32, 3524578, 263130836

Finally, let's store those values into a list (of lists). 

We could do it this way:

In [12]:
data = []

with open(file_name) as f:
    next(f)  # skip header row
    for row in f:
        data.append(list(map(int, row.strip().split(','))))
        
print(data[:10])

[[0, 1, 1, 1], [1, 1, 1, 1], [2, 2, 2, 2], [3, 3, 6, 3], [4, 5, 24, 1], [5, 8, 120, 8], [6, 13, 720, 1], [7, 21, 5040, 21], [8, 34, 40320, 2], [9, 55, 362880, 5]]


But, we can also just use a comprehension:

In [13]:
with open(file_name) as f:
    next(f)  # skip header row
    data = [list(map(int, row.strip().split(','))) for row in f]
        
print(data[:10])

[[0, 1, 1, 1], [1, 1, 1, 1], [2, 2, 2, 2], [3, 3, 6, 3], [4, 5, 24, 1], [5, 8, 120, 8], [6, 13, 720, 1], [7, 21, 5040, 21], [8, 34, 40320, 2], [9, 55, 362880, 5]]


Now that we have our pre-calculated data, let's create individual sequences for Fibonacci, factorial and gcd numbers:

In [14]:
fib_stored = [row[1] for row in data]
fact_stored = [row[2] for row in data]
gcd_stored = [row[3] for row in data]

Finally, we can write our functions, starting with `fact`:

In [15]:
def fact(i):
    if i < len(fact_stored):
        print('looking up fact in cache...')
        return fact_stored[i]
    else:
        return factorial(i)

In [16]:
fact(10)

looking up fact in cache...


3628800

We can do something similar for Fibonacci numbers:

In [17]:
def fib(i):
    if i < len(fib_stored):
        print('looking up fib in cache...')
        return fib_stored[i]
    else:
        # not cached - so we need to calculate it
        if i in {0, 1}:
            return 1
        else:
            return fib(i-1) + fib(i-2)

Before we run this, let's make sure we apply an `lru_cache` to it as well:

In [18]:
@lru_cache
def fib(i):
    if i < len(fib_stored):
        print('looking up in cache...')
        return fib_stored[i]
    else:
        # not cached - so we need to calculate it
        if i in {0, 1}:
            return 1
        else:
            return fib(i-1) + fib(i-2)

In [19]:
fib(10)

looking up in cache...


89

In [20]:
fib(101)

looking up in cache...
looking up in cache...


927372692193078999176

You'll notice that to calculate `fib(101)` required calculating `fib(100) + fib(99)`, which were both in our loaded data - hence why we see two `looking up fib in cache...` prints in our output.

Finally, we can process `gcd` in the same way:

In [21]:
def gcd_fib_fact(i):
    if i < len(gcd_stored):
        print('Looking up gcd in cache...')
        return gcd_stored[i]
    else:
        return gcd(fact(i), fib(i))

In [22]:
gcd_fib_fact(11)

Looking up gcd in cache...


144

In [23]:
gcd_fib_fact(101)

8